# Tugas Besar B
IF3270 Pembelajaran Mesin<br>
Backward Propagation - Mini Batch Gradient Descent

Developed by:
1. K01 13520010 - Ken Kalang Al Qalyubi
2. K01 13520036 - I Gede Arya Raditya Parameswara
3. K02 13520061 - Gibran Darmawan
4. K03 13520119 - Marchotridyo

## Main Program

### Library

In [3]:
import json, math
import networkx as nx
import matplotlib.pyplot as plt
from enum import Enum
import numpy as np
import random

### Enum

In [4]:
class LayerEnum(Enum):
    INPUT = "INPUT"
    HIDDEN = "HIDDEN"
    OUTPUT = "OUTPUT"

class ActivationFuncEnum(Enum):
    SIGMOID = "SIGMOID"
    LINEAR = "LINEAR"
    RELU = "RELU"
    SOFTMAX = "SOFTMAX"

### File Utility

In [5]:
class FileUtility:
    @staticmethod
    def import_json(file_name):
        with open(file_name) as json_file:
            return json.load(json_file)

    @staticmethod
    def export_json(file_name, data):
        with open(file_name, 'w') as outfile:
            json.dump(data, outfile)

### Activation

In [6]:
class Activation:
    @staticmethod
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    
    @staticmethod
    def linear(x):
        return x
    
    @staticmethod
    def relu(x):
        return np.maximum(0, x)
    
    @staticmethod
    def softmax(x):
        return np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)

    @staticmethod
    def derivative_sigmoid(x):
        return Activation.sigmoid(x) * (1 - Activation.sigmoid(x))
    
    @staticmethod
    def derivative_linear( x):
        return 1
    
    @staticmethod
    def derivative_relu( x):
        return 1 if x > 0 else 0
    
    @staticmethod
    def derivative_softmax( x):
        return -(1 - x)
    

### Layer

In [7]:
class Layer:
    # Layer adalah kelas yang menyimpan sejumlah neutron berikut fungsi aktivasinya
    def __init__(self, neurons: list, type: str, activation_func: str):
        self.__neurons = neurons
        self.__type = type
        self.__activation_func = activation_func

    def add_neuron(self, neuron):
        self.__neurons.append(neuron)

    def get_neurons(self):
        return self.__neurons
    
    def get_type(self):
        return self.__type
    
    def get_activation_func(self):
        return self.__activation_func

### Error Counter

In [8]:
class ErrorCounter:
    @staticmethod
    def sum_squared_error(y_true: np.ndarray, y_pred: np.ndarray) -> float:
        return np.sum((y_true - y_pred) ** 2)

### Neuron

In [9]:
class Neuron:
    def __init__(
        self, 
        layer: Layer,
        weight: list,
        bias: float, 
    ):
        self.__layer: Layer = layer
        self.__weight: list = weight
        self.__bias: float = bias
        self.__net: float = 0.0
        self.__value: float = 0.0

    def activate(self):
        if self.__layer.get_activation_func() == ActivationFuncEnum.SIGMOID.value:
            self.__value = 1 / (1 + math.exp(-self.__net))
        elif self.__layer.get_activation_func() == ActivationFuncEnum.LINEAR.value:
            self.__value = self.__net
        elif self.__layer.get_activation_func() == ActivationFuncEnum.RELU.value:
            self.__value = max(0, self.__net)
        elif self.__layer.get_activation_func() == ActivationFuncEnum.SOFTMAX.value:
            layer_neurons: list = self.__layer.get_neurons()
            exp_sum: float = 0.0

            for neuron in layer_neurons:
                exp_sum += math.exp(neuron.get_net())

            self.__value = math.exp(self.__net) / exp_sum

    def set_value(self, value):
        self.__value = value

    def set_net(self, net):
        self.__net = net

    def set_weight(self, index, weight):
        self.__weight[index] = weight

    def get_value(self):
        return self.__value

    def get_net(self):
        return self.__net

    def get_weight(self, index):
        return self.__weight[index]

    def get_bias(self):
        return self.__bias

### ANN Graph

In [10]:
class ANNGraph:
    def __init__(self, layer_size: int, neuron_sizes: list, activation_func: list):
        self.__layer_size = layer_size
        self.__neuron_sizes = neuron_sizes
        self.__activation_func = activation_func
        self.__layers  = []
        self.__outout_activation_func = activation_func[-1]

        self.__build_ann_graph()
        
    def train(self, data: np.ndarray, target: np.ndarray ,learning_rate: float, error_threshold: float, max_epoch: int, batch_size: int = 50):
        self.__reset_weights()

        batches_x = []
        batches_y = []
        
        batches_x = [data[i:i+batch_size] for i in range(0, len(data), batch_size)]
        batches_y = [target[i:i+batch_size] for i in range(0, len(target), batch_size)]
        
        batches_len = len(batches_x)
        
        epoch = 0
        while epoch < max_epoch:
            error = 0
            random_batches_x = []
            random_batches_y = []

            random_index = random.sample(range(batches_len), batches_len)
            for i in random_index:
                random_batches_x.append(batches_x[i])
                random_batches_y.append(batches_y[i])

            for i in range(batches_len):
                batch_x = random_batches_x[i]
                batch_y = random_batches_y[i]

                batch_error = 0

                for j in range(len(batch_x)):
                    self.__feed_forward(batch_x[j])
                    batch_error += self.__error(batch_y[j])
                    self.__back_propagation(batch_y[j], learning_rate)

                batch_error /= len(batch_x)
                error += batch_error
            
            error /= batches_len

            print(error)

            epoch += 1
            if error < error_threshold:
                print("berhenti pada epoch ke-", epoch)
                break
            if epoch % 1000 == 0:
                print("epoch ke-", epoch, "error: ", error)

        if epoch == max_epoch:
            print(error)
            print("Mencapai batas epoch")
        return                
    
    def __error(self, target: np.array):
        error = 0
        print("target", target)

        # sesuaikan dengan nilai hasil sebenarnya pada dataset
        sample_target = [target[0] for i in range(len(self.__layers[-1].get_neurons()))]

        if self.__outout_activation_func == ActivationFuncEnum.SOFTMAX.value:
            epsilon = 1e-15
            predictions = [max(epsilon, min(1 - epsilon, neuron.get_value())) for neuron in self.__layers[-1].get_neurons()]

            for i in range(len(sample_target)):
                error += -math.log(predictions[i])
            error /= len(sample_target)

            return error

        for i in range(len(sample_target)):
            error += (self.__layers[-1].get_neurons()[i].get_value() - sample_target[i]) ** 2
        error /= 2
        return error

    def __feed_forward(self, sample: np.array):
        for i in range(self.__layer_size):
            if i == 0:
                for j in range(self.__neuron_sizes[i]):
                    self.__layers[i].get_neurons()[j].set_value(sample[j])
            else:
                for j in range(self.__neuron_sizes[i]):
                    net = 0
                    for k in range(self.__neuron_sizes[i - 1]):
                        net += self.__layers[i - 1].get_neurons()[k].get_value() * self.__layers[i].get_neurons()[j].get_weight(k)
                    net += self.__layers[i].get_neurons()[j].get_bias()

                    self.__layers[i].get_neurons()[j].set_net(net)
                    self.__layers[i].get_neurons()[j].activate()
        return self.__layers         
    
    def __back_propagation(self, target: np.ndarray, learning_rate: float):
        output_layer = self.__layers[-1]
        output_layer_neurons = output_layer.get_neurons()
        output_delta = []
        for i in range(len(output_layer_neurons)):
            sample_target = [target[0] for i in range(len(self.__layers[-1].get_neurons()))] 
            neuron = output_layer_neurons[i]
            if self.__outout_activation_func == ActivationFuncEnum.LINEAR.value:
                output_delta.append(Activation.derivative_linear(neuron.get_net()) * ((sample_target[i] - neuron.get_value()) ** 2))
            if self.__outout_activation_func == ActivationFuncEnum.SIGMOID.value:
                output_delta.append(Activation.derivative_sigmoid(neuron.get_net()) * ((sample_target[i] - neuron.get_value()) ** 2))
            if self.__outout_activation_func == ActivationFuncEnum.RELU.value:
                output_delta.append(Activation.derivative_relu(neuron.get_net()) * ((sample_target[i] - neuron.get_value()) ** 2))
            if self.__outout_activation_func == ActivationFuncEnum.SOFTMAX.value:
                output_delta.append(Activation.derivative_softmax(neuron.get_value()))

        for i in range(self.__layer_size - 2, 0, -1):
            layer = self.__layers[i]
            layer_neurons = layer.get_neurons()
            next_layer = self.__layers[i + 1]
            next_layer_neurons = next_layer.get_neurons()

            delta = []
            for j in range(len(layer_neurons)):
                neuron = layer_neurons[j]
                delta_sum = 0
                for k in range(len(next_layer_neurons)):
                    next_neuron = next_layer_neurons[k]
                    delta_sum += next_neuron.get_weight(j) * output_delta[k]
                if layer.get_activation_func() == ActivationFuncEnum.LINEAR.value:
                    delta.append(Activation.derivative_linear(neuron.get_net()) * delta_sum)
                if layer.get_activation_func() == ActivationFuncEnum.SIGMOID.value:
                    delta.append(Activation.derivative_sigmoid(neuron.get_net()) * delta_sum)
                if layer.get_activation_func() == ActivationFuncEnum.RELU.value:
                    delta.append(Activation.derivative_relu(neuron.get_net()) * delta_sum)
                if layer.get_activation_func() == ActivationFuncEnum.SOFTMAX.value:
                    delta.append(Activation.derivative_softmax(neuron.get_value()))

            for j in range(len(layer_neurons)):
                neuron = layer_neurons[j]
                for k in range(len(next_layer_neurons)):
                    next_neuron = next_layer_neurons[k]
                    delta_weight = learning_rate * output_delta[k] * neuron.get_value()
                    new_weight = next_neuron.get_weight(j) + delta_weight
                    next_neuron.set_weight(j, new_weight)

            output_delta = delta


        input_layer = self.__layers[0]
        input_layer_neurons = input_layer.get_neurons()
        next_layer = self.__layers[1]
        next_layer_neurons = next_layer.get_neurons()

        for i in range(len(input_layer_neurons)):
            neuron = input_layer_neurons[i]
            for j in range(len(next_layer_neurons)):
                next_neuron = next_layer_neurons[j]
                delta_weight = learning_rate * output_delta[j] * neuron.get_value()
                new_weight = next_neuron.get_weight(i) + delta_weight
                next_neuron.set_weight(i, new_weight)
    
    def __build_ann_graph(self):
        for i in range(self.__layer_size):
            if i == 0:
                layer = Layer([], LayerEnum.INPUT.value, self.__activation_func[i])
            elif i == (self.__layer_size - 1):
                layer = Layer([], LayerEnum.OUTPUT.value, self.__activation_func[i])
            else:
                layer = Layer([], LayerEnum.HIDDEN.value, self.__activation_func[i])

            for j in range(self.__neuron_sizes[i]):
                weight = []
                
                if i > 0:
                    for k in range(self.__neuron_sizes[i - 1] + 1):
                        weight.append(np.random.uniform(-0.5, 0.5))
                
                neuron = Neuron(layer, weight, 1)
                layer.add_neuron(neuron)
            
            self.__layers.append(layer)

        return self.__layers
    
    def __reset_weights(self):
        for i in range(self.__layer_size):
            if i == 0:
                continue
            for j in range(self.__neuron_sizes[i]):
                for k in range(self.__neuron_sizes[i - 1] + 1):
                    self.__layers[i].get_neurons()[j].set_weight(k, np.random.uniform(-0.5, 0.5))
    
    def draw_ann_graph(self):
        # Terminologies:
        # Xi = neuron ke-i di input layer
        # Hij = neuron ke-j di hidden layer ke-i
        # Oi = neuron ke-i di output layer

        G = nx.DiGraph()

        # Proses setiap layer
        for i, layer in enumerate(self.layers):
            if i == 0:
                continue

            prev_layer = self.layers[i - 1]
            prev_prefix = ""
            prefix = ""

            if prev_layer.get_type() == LayerEnum.INPUT.value:
                prev_prefix = "X"
            elif prev_layer.get_type() == LayerEnum.HIDDEN.value:
                prev_prefix = f"H{i - 1}"
            else:
                prev_prefix = "O"
            
            if layer.get_type() == LayerEnum.INPUT.value:
                prefix = "X"
            elif layer.get_type() == LayerEnum.HIDDEN.value:
                prefix = f"H{i}"
            else:
                prefix = "O"

            # Tambahkan edge dari setiap neuron di prev_layer ke layer
            for j, _ in enumerate(prev_layer.get_neurons()):
                for k, neuron in enumerate(layer.get_neurons()):
                    if j == 0:
                        print(f"Bobot bias untuk {prefix}{k + 1} = {neuron.get_weight(0)}")
                    G.add_edge(f"{prev_prefix}{j + 1}", f"{prefix}{k + 1}", weight=neuron.get_weight(j + 1))
            
        # Set posisi node graph
        pos = {}
        curr_x = 0
        for i, layer in enumerate(self.layers):
            curr_y = 0

            prefix = ""
            if layer.get_type() == LayerEnum.INPUT.value:
                prefix = "X"
            elif layer.get_type() == LayerEnum.HIDDEN.value:
                prefix = f"H{i}"
            else:
                prefix = "O"
            
            for j, _ in enumerate(layer.get_neurons()):
                pos[f"{prefix}{j + 1}"] = (curr_x, curr_y)
                curr_y += 1

            curr_x += 1

        options = {
            "font_size": 12,
            "node_size": 2000,
            "node_color": "white",
            "edgecolors": "black",
            "linewidths": 5,
            "width": 5,
        }

        nx.draw_networkx(G, pos, **options)
        edge_labels = nx.get_edge_attributes(G, "weight")
        nx.draw_networkx_edge_labels(G, pos, edge_labels, label_pos=0.6)

        ax = plt.gca()
        ax.margins(0.2)
        plt.axis("off")
        plt.show()

## Testing

In [11]:
from sklearn import datasets
from sklearn.neural_network import MLPClassifier
iris = datasets.load_iris()
x, y = iris.data, iris.target

graph = ANNGraph(3, [len(iris.feature_names), 2, len(iris.target_names)], [None, ActivationFuncEnum.SIGMOID.value, ActivationFuncEnum.SOFTMAX.value])

In [12]:
graph.train(x, y.reshape(-1,1), 1e-2, 0.1, 1000, 50)

target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [0]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]
target [1]